In [1]:
from openai import OpenAI
from datetime import datetime
import json
import os
import random
from dotenv import load_dotenv
import os

client = OpenAI(
    # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx",
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",  # 填写DashScope SDK的base_url
)

In [5]:
# 定义工具列表，模型在选择使用哪个工具时会参考工具的name和description
tools = [

    # 工具2 获取指定城市的天气
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "当你想查询指定城市的天气时非常有用。",
            "parameters": {
                "type": "object",
                "properties": {
                    # 查询天气时需要提供位置，因此参数设置为location
                    "location": {
                        "type": "string",
                        "description": "城市或县区，比如北京市、杭州市、余杭区等。",
                    }
                },
                "required": ["location"],
            },
        },
    },
]

In [14]:
# 模拟天气查询工具。返回结果示例：“北京今天是雨天。”
def get_current_weather(arguments):
    # 定义备选的天气条件列表
    weather_conditions = ["晴天", "多云", "雨天"]
    # 随机选择一个天气条件
    random_weather = random.choice(weather_conditions)
    # 从 JSON 中提取位置信息
    location = arguments["location"]
    # 返回格式化的天气信息
    return f"{location}今天是{random_weather}。"

In [2]:
messages = [
    {
        "role": "system",
        "content": """你是一个很有帮助的助手。如果用户提问关于天气的问题，请调用 ‘get_current_weather’ 函数;
     如果用户提问关于时间的问题，请调用‘get_current_time’函数。
     请以友好的语气回答问题。""",
    },
    {
        "role": "user",
        "content": "上海天气"
    }
]
print("messages 数组创建完成\n")

messages 数组创建完成



In [7]:
def function_calling():
    completion = client.chat.completions.create(
        model="qwen-plus",  # 此处以qwen-plus为例，可按需更换模型名称。模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
        messages=messages,
        tools=tools
    )
    print("返回对象：")
    print(completion.choices[0].message.model_dump_json())
    print("\n")
    return completion


print("正在发起function calling...")
completion = function_calling()
print(completion)

正在发起function calling...
返回对象：
{"content":"","refusal":null,"role":"assistant","annotations":null,"audio":null,"function_call":null,"tool_calls":[{"id":"call_309e7d3ca66948309df031","function":{"arguments":"{\"location\": \"上海\"}","name":"get_current_weather"},"type":"function","index":0}]}


ChatCompletion(id='chatcmpl-118b5890-b89a-95aa-97ca-8ae8f148c262', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_309e7d3ca66948309df031', function=Function(arguments='{"location": "上海"}', name='get_current_weather'), type='function', index=0)]))], created=1755003241, model='qwen-plus', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=20, prompt_tokens=230, total_tokens=250, completion_tokens_details=None, prompt_tokens_details=PromptTokensDetail

In [ ]:
import json
funtion_name = completion.choices[0].message.tool_calls[0].function.name
arguments_string= json.loads(completion.choices[0].message.tool_calls[0].function.arguments)

# LLM返回的参数是字符串，需要用 json.loads() 将其转换为Python字典


# 创建函数映射表
function_map = {
    "get_current_weather": get_current_weather


}

# 获取函数实体

funciton = function_map[funtion_name]
if arguments_string == {}:
    funciton_output = funciton()
else:
    function_output = function(arguments_string)
    # 打印工具的输出
print(f"工具函数输出：{function_output}\n")


工具函数输出：上海今天是晴天。

